# Kshitij Sonje CS24MTECH11025

# Assignment 1

In [26]:
import csv
import numpy as np
from scipy.linalg import null_space

In [27]:
# Data parsing utility functions
def extract_objective_data(raw_data):
    start_point = np.array([float(val) for val in raw_data[0][:-1]])
    cost_coeffs = np.array([float(val) for val in raw_data[1][:-1]])
    return start_point, cost_coeffs

def extract_constraint_bounds(raw_data):
    return np.array([float(row[-1]) for row in raw_data[2:]])

def extract_constraint_matrix(raw_data):
    return np.array([[float(val) for val in row[:-1]] for row in raw_data[2:]])

In [28]:
#Read the CSV
def parse_csv_input(filename, has_initial_point=True):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        raw_data = list(reader)
    
    start_point, cost_coeffs = extract_objective_data(raw_data)
    bounds = extract_constraint_bounds(raw_data)
    constraint_mat = extract_constraint_matrix(raw_data)
    
    return constraint_mat, start_point, bounds, cost_coeffs

In [29]:
# Constraint analysis utilities
def compute_constraint_products(coeff_matrix, current_pt, bounds, tolerance=1e-8):
    constraint_vals = np.dot(coeff_matrix, current_pt)
    active_mask = np.abs(constraint_vals - bounds) < tolerance
    return active_mask

def separate_constraint_sets(coeff_matrix, active_mask):
    active_constraints = coeff_matrix[active_mask]
    inactive_constraints = coeff_matrix[~active_mask]
    return active_constraints, inactive_constraints

In [30]:
#Find tight mask
def analyze_constraints(coeff_matrix, current_pt, bounds, tolerance=1e-8):
    active_mask = compute_constraint_products(coeff_matrix, current_pt, bounds, tolerance)
    active_set, inactive_set = separate_constraint_sets(coeff_matrix, active_mask)
    return active_mask, active_set, inactive_set

In [31]:
# Movement direction calculation function
def invert_and_negate_matrix(active_matrix):
    matrix_inverse = np.linalg.inv(active_matrix)
    negated_inverse = -1 * matrix_inverse
    return negated_inverse

def handle_singular_matrix():
    print("Matrix is singular. Cannot compute the inverse.")
    return None


In [32]:
#Get Directions
def compute_movement_vectors(active_matrix):
    try:
        return invert_and_negate_matrix(active_matrix)
    except np.linalg.LinAlgError:
        return handle_singular_matrix()

In [33]:
# Phase 1: Feasible point to vertex conversion helper functions
def calculate_constraint_rank(active_constraints):
    if len(active_constraints) == 0:
        return 0
    else:
        return np.linalg.matrix_rank(active_constraints)

def check_vertex_status(current_rank, required_dimension):
    return current_rank == required_dimension

def find_movement_direction(active_constraints, inactive_constraints):
    if len(active_constraints) == 0:
        np.random.seed(42)
        direction = np.random.rand(inactive_constraints.shape[-1])
        print(f"Using deterministic random direction: {direction}")
    else:
        null_space_matrix = null_space(active_constraints)
        direction = null_space_matrix[:, 0]
        print(f"Using null space direction: {direction}")
    return direction

def compute_step_size(bounds_vector, constraint_mask, inactive_constraints, current_position, move_direction):
    step_ratios = [(bound_val - np.dot(constraint_row, current_position)) / np.dot(constraint_row, move_direction) 
                   for bound_val, constraint_row in zip(bounds_vector[~constraint_mask], inactive_constraints)]
    positive_ratios = [ratio for ratio in step_ratios if ratio > 0]
    optimal_step = min(positive_ratios)
    print(f"Step size alpha: {optimal_step}")
    return optimal_step

def update_constraint_analysis(constraint_matrix, new_position, bounds_vector):
    updated_mask, updated_active, updated_inactive = analyze_constraints(constraint_matrix, new_position, bounds_vector)
    return updated_mask, updated_active, updated_inactive

def iterate_to_vertex(constraint_matrix, bounds_vector, current_pos, objective_coeffs, problem_dimension,
                      constraint_mask, active_constraints, inactive_constraints, current_rank, cost_history, position_history):
    max_iterations = 1000
    for iteration_count in range(1, max_iterations + 1):
        if current_rank == problem_dimension:
            break
            
        print(f"Iteration: {iteration_count} - Rank: {current_rank}")
        
        move_direction = find_movement_direction(active_constraints, inactive_constraints)
        step_size = compute_step_size(bounds_vector, constraint_mask, inactive_constraints, current_pos, move_direction)
        
        new_position = current_pos + step_size * move_direction
        print(f"Moving from {current_pos} to {new_position}")
        
        constraint_mask, active_constraints, inactive_constraints = update_constraint_analysis(constraint_matrix, new_position, bounds_vector)
        current_pos = new_position
        current_rank = calculate_constraint_rank(active_constraints)
        
        cost_history.append(np.dot(objective_coeffs, new_position))
        position_history.append(new_position)
    
    return current_pos, constraint_mask, active_constraints, inactive_constraints, current_rank

def locate_initial_vertex(constraint_matrix, bounds_vector, start_position, objective_coeffs, problem_dimension):
    cost_history = []
    position_history = []
    
    cost_history.append(np.dot(objective_coeffs, start_position))
    position_history.append(start_position)
    
    constraint_mask, active_constraints, inactive_constraints = analyze_constraints(constraint_matrix, start_position, bounds_vector)
    current_rank = calculate_constraint_rank(active_constraints)
    
    if check_vertex_status(current_rank, problem_dimension):
        print("Already at a vertex!")
        return start_position, cost_history, position_history
    else:
        print("Not at a vertex. Searching for a vertex...")
        
        current_pos = start_position
        iteration_count = 0
        
        current_pos, constraint_mask, active_constraints, inactive_constraints, current_rank = iterate_to_vertex(
            constraint_matrix, bounds_vector, current_pos, objective_coeffs, problem_dimension, 
            constraint_mask, active_constraints, inactive_constraints, current_rank, cost_history, position_history)
        
        print(f"Vertex found! - Rank: {current_rank}")
        return current_pos, cost_history, position_history

In [34]:
# Phase 2: Vertex-to-vertex optimization helper functions
def display_iteration_info(iteration_num, current_vertex):
    print(f"\nIteration: {iteration_num}")
    print(f"Current vertex: {current_vertex}")

def find_improving_directions(movement_directions, objective_coeffs):
    improving_dirs = []
    direction_idx = 0
    while direction_idx < len(movement_directions):
        current_dir = movement_directions[direction_idx]
        improvement_value = np.dot(current_dir, objective_coeffs)
        print(f"Direction {direction_idx}: {current_dir}, c·u = {improvement_value}")
        if improvement_value > 0:
            improving_dirs.append(current_dir)
        direction_idx += 1
    return improving_dirs

def calculate_optimal_step(bounds_vec, constraint_mask, inactive_set, current_vertex, selected_direction):
    step_candidates = [(bound_val - np.dot(constraint_row, current_vertex)) / np.dot(constraint_row, selected_direction) 
                       for bound_val, constraint_row in zip(bounds_vec[~constraint_mask], inactive_set)]
    valid_steps = [step for step in step_candidates if step > 0]
    print(f"All positive step sizes: {valid_steps}")
    print(f"Number of positive step sizes: {len(valid_steps)}")
    optimal_step_size = min(valid_steps)
    print(f"Selected step size: {optimal_step_size}")
    return optimal_step_size

def execute_vertex_iteration(iteration_num, current_vertex, constraint_matrix, bounds_vec, objective_coeffs):
    display_iteration_info(iteration_num, current_vertex)
    
    constraint_mask, active_set, inactive_set = analyze_constraints(constraint_matrix, current_vertex, bounds_vec)
    print(f"Active constraints: {np.sum(constraint_mask)} out of {len(constraint_mask)}")
    print(f"Active constraint matrix:\n{active_set}")
    
    movement_directions = compute_movement_vectors(active_set).T
    print(f"Available movement directions: {len(movement_directions)}")
    print(f"Direction matrix:\n{movement_directions}")
    
    improving_directions = find_improving_directions(movement_directions, objective_coeffs)
    print(f"Number of improving directions: {len(improving_directions)}")
    
    if not improving_directions:
        print("Reached the optimal vertex!")
        return None, None, None, None, True
    
    return constraint_mask, active_set, inactive_set, improving_directions, False

def march_to_optimal_vertex(constraint_matrix, bounds_vec, start_vertex, objective_coeffs, problem_dims):
    cost_track = []
    vertex_track = []
    
    current_vertex = start_vertex
    cost_track.append(np.dot(objective_coeffs, current_vertex))
    vertex_track.append(current_vertex)
    
    print(f"Starting vertex-to-vertex from: {current_vertex} with cost: {np.dot(objective_coeffs, current_vertex)}")
    
    max_vertex_iterations = 100
    for iteration_num in range(1, max_vertex_iterations + 1):
        constraint_mask, active_set, inactive_set, improving_directions, is_optimal = execute_vertex_iteration(
            iteration_num, current_vertex, constraint_matrix, bounds_vec, objective_coeffs)
        
        if is_optimal:
            return current_vertex, cost_track, vertex_track
        
        selected_direction = improving_directions[0]
        print(f"Selected direction: {selected_direction}")
        
        step_size = calculate_optimal_step(bounds_vec, constraint_mask, inactive_set, current_vertex, selected_direction)
        
        next_vertex = current_vertex + step_size * selected_direction
        print(f"Moving to: {next_vertex}")
        current_vertex = next_vertex
        
        new_cost = np.dot(objective_coeffs, next_vertex)
        cost_track.append(new_cost)
        vertex_track.append(next_vertex)
        print(f"New cost: {new_cost}")
    
    return current_vertex, cost_track, vertex_track

In [35]:
# Load problem data from CSV file
constraint_mat, start_point, bounds_vec, objective_coeffs = parse_csv_input('input.csv', has_initial_point=True)

problem_rows, problem_cols = len(bounds_vec), len(objective_coeffs)

# Display extracted data
print("Initial Feasible Point:", start_point)
print("Objective Coefficients:", objective_coeffs)
print("Constraint Bounds:", bounds_vec)
print("Constraint Matrix:")
print(constraint_mat)
print(f"Problem size: {problem_rows} constraints, {problem_cols} variables")

Initial Feasible Point: [4. 1.]
Objective Coefficients: [1. 1.]
Constraint Bounds: [ 9. -1. 25. -9.  1. 15.]
Constraint Matrix:
[[ 0.  1.]
 [ 0. -1.]
 [ 2.  1.]
 [-2. -1.]
 [-2.  1.]
 [ 2. -1.]]
Problem size: 6 constraints, 2 variables


In [36]:
z_new, z_cost_all, z_all = locate_initial_vertex(constraint_mat, bounds_vec, start_point, objective_coeffs, problem_cols)

print(z_new)
print(z_all)
print(z_cost_all)

z_optimal, z_cost_all, z_all = march_to_optimal_vertex(constraint_mat, bounds_vec, z_new, objective_coeffs, problem_cols)

print(z_optimal)
print(z_all)
print(z_cost_all)

Already at a vertex!
[4. 1.]
[array([4., 1.])]
[np.float64(5.0)]
Starting vertex-to-vertex from: [4. 1.] with cost: 5.0

Iteration: 1
Current vertex: [4. 1.]
Active constraints: 2 out of 6
Active constraint matrix:
[[ 0. -1.]
 [-2. -1.]]
Available movement directions: 2
Direction matrix:
[[-0.5  1. ]
 [ 0.5  0. ]]
Direction 0: [-0.5  1. ], c·u = 0.5
Direction 1: [0.5 0. ], c·u = 0.5
Number of improving directions: 2
Selected direction: [-0.5  1. ]
All positive step sizes: [np.float64(8.0), np.float64(inf), np.float64(4.0)]
Number of positive step sizes: 3
Selected step size: 4.0
Moving to: [2. 5.]
New cost: 7.0

Iteration: 2
Current vertex: [2. 5.]
Active constraints: 2 out of 6
Active constraint matrix:
[[-2. -1.]
 [-2.  1.]]
Available movement directions: 2
Direction matrix:
[[ 0.25  0.5 ]
 [ 0.25 -0.5 ]]
Direction 0: [0.25 0.5 ], c·u = 0.75
Direction 1: [ 0.25 -0.5 ], c·u = -0.25
Number of improving directions: 1
Selected direction: [0.25 0.5 ]
All positive step sizes: [np.float64(8

/tmp/ipykernel_10318/3465075593.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  step_candidates = [(bound_val - np.dot(constraint_row, current_vertex)) / np.dot(constraint_row, selected_direction)


In [37]:
# Vertex Marching Sequence
print("VERTEX MARCHING SEQUENCE:")

step_counter = 1
while step_counter <= len(z_all):
    vertex = z_all[step_counter - 1]
    cost = z_cost_all[step_counter - 1]
    if step_counter == len(z_all):
        print(f"Step {step_counter}: Vertex → {vertex} (Cost: {cost:.4f})")
    else:
        print(f"Step {step_counter}: Vertex → {vertex} (Cost: {cost:.4f})")
    step_counter += 1

print(f"\nOptimal Solution: {z_optimal} with cost {np.dot(objective_coeffs, z_optimal):.4f}")

VERTEX MARCHING SEQUENCE:
Step 1: Vertex → [4. 1.] (Cost: 5.0000)
Step 2: Vertex → [2. 5.] (Cost: 7.0000)
Step 3: Vertex → [4. 9.] (Cost: 13.0000)
Step 4: Vertex → [8. 9.] (Cost: 17.0000)

Optimal Solution: [8. 9.] with cost 17.0000
